In [1]:
# 1. 스파크 세션(SparkSession) 만들기
from pyspark.sql import SparkSession

# "restaurant-review-average"이라는 이름으로 스파크 프로그램 시작
#  스파크를 내 컴퓨터에서(local) 실행하겠다는 의미입니다.
spark = SparkSession.builder \
    .appName("restaurant-review-average") \
    .master("local[*]") \
    .getOrCreate()

In [2]:
sc = spark.sparkContext  # 필요 시 RDD API를 위해 사용

# Reduce
RDD.reduce(<func>)
사용자가 지정하는 함수를 받아 여러 개의 값을 하나로 줄여줍니다.

In [3]:
from operator import add

In [10]:
sample_rdd = sc.parallelize([1, 2, 3, 4, 5])
sample_rdd

ParallelCollectionRDD[7] at readRDDFromFile at PythonRDD.scala:289

In [13]:
#RDD에 있는 숫자의 합계를 계산: [1, 2, 3, 4, 5] =15
sample_rdd.reduce(add)

15

In [6]:
sample_rdd = sc.parallelize([1, 2, 3, 4])
sample_rdd.reduce(lambda x, y : (x * 2)+ y)

18

In [ ]:
sc.parallelize([1, 2, 3, 4], 2).reduce(lambda x, y : (x * 2) + y)

In [ ]:
# 파티션을 3개로 지정

In [14]:
sc.parallelize([1, 2, 3, 4], 3).reduce(lambda x, y : (x * 2) + y)

18

In [15]:
sc.parallelize([1, 2, 3, 4], 1).reduce(lambda x, y : (x * 2) + y)

26

In [ ]:
# Fold
RDD.fold(zeroValue, <func>)
reduce와 비슷하지만, zeroValue에 넣어놓고 싶은 시작값을 지정해서 reduce가 가능
zeroValue는 파티션 마다 계산이 일어날 때 하나씩 더해지는 값
fold()를 사용하여 합계를 구하되, 초기값을 지정

In [ ]:
rdd = sc.parallelize([2, 3, 4], 4)

#숫자의 곱을 계산해서 합한다
print(rdd.reduce(lambda x, y : (x * y))) # 2 x 3 -> x 4
print(rdd.fold(1, lambda x , y : (x * y))) # 1 x 2 x 3 x 4

In [ ]:
rdd.fold(2, lambda x , y : (x * y)) 
# 초기값 2 지정 :: 2x2 , 2x3, 2x4, "" > 4 , 6, 8, "" 
# 다시 초기값 2 지정 :: 4x2*6, 48x2*8, "" > 768

In [ ]:
print(rdd.reduce(lambda x, y : x + y)) 
print(rdd.fold(1, lambda x, y : x + y)) 

In [ ]:
# 대규모 숫자 데이터셋 생성
numbers_rdd = sc.parallelize(range(1, 1000001))

# fold를 사용하여 총합계 계산
total_sum = numbers_rdd.fold(0, lambda x, y: x + y)

print(f"Total sum: {total_sum}")

In [ ]:
# 직원 급여 데이터
salaries_rdd = sc.parallelize([50000, 60000, 55000, 75000, 65000, 80000])

# fold를 사용하여 최대 급여 찾기
max_salary = salaries_rdd.fold(0, lambda x, y: max(x, y))

print(f"Maximum salary: {max_salary}")

In [ ]:
# 부서별 직원 급여 데이터
dept_salaries = [("IT", 50000), ("HR", 45000), ("IT", 60000), 
                 ("Finance", 55000), ("HR", 50000), ("IT", 65000)]
dept_rdd = sc.parallelize(dept_salaries)

# foldByKey를 사용하여 부서별 최대 급여 찾기
max_salary_by_dept = dept_rdd.foldByKey(0, lambda x, y: max(x, y))

print("Maximum salary by department:")
for dept, max_salary in max_salary_by_dept.collect():
    print(f"{dept}: {max_salary}")

# GroupBy
RDD.groupBy(<func>)
그룹핑 함수를 받아 reduction
groupBy() 메서드는 주어진 함수를 기준으로 RDD의 요소들을 그룹화합니다.

In [16]:
rdd = sc.parallelize([1, 1, 2, 3, 5, 8])
result = rdd.groupBy(lambda x : x % 2).collect()

sorted([(x, sorted(y)) for (x, y) in result])

[(0, [2, 8]), (1, [1, 1, 3, 5])]

In [ ]:
[(x, sorted(y)) for (x, y) in result]

In [17]:
#판매 데이터 분석 (카테고리별 총 매출 계산)
sales_data = [
    ("Electronics", 1000),
    ("Clothing", 500),
    ("Electronics", 1500),
    ("Books", 300),
    ("Clothing", 750),
    ("Books", 200)
]

In [18]:
# RDD 생성
sales_rdd = sc.parallelize(sales_data)

# 카테고리별 총 매출 계산
category_sales = sales_rdd.groupBy(lambda x: x[0]) \
    .mapValues(lambda values: sum(item[1] for item in values)) \
    .collect()

# 결과 출력
for category, total_sales in category_sales:
    print(f"{category}: ${total_sales}")

Clothing: $1250
Electronics: $2500
Books: $500


In [19]:
#로그 데이터 처리 (IP 주소별 접속 횟수 분석)
# SparkContext 생성 가정 (sc)
log_data = [
    ("192.168.1.1", "login"),
    ("192.168.1.2", "purchase"),
    ("192.168.1.1", "view"),
    ("192.168.1.3", "login"),
    ("192.168.1.2", "login"),
    ("192.168.1.1", "purchase")
]

# RDD 생성
log_rdd = sc.parallelize(log_data)


In [36]:
category_log = log_rdd.groupBy(lambda x: x[0]) \
    .mapValues(lambda values: [for item in values]) \
    .collect()
category_log

[('192.168.1.1', ['login', 'view', 'purchase']),
 ('192.168.1.2', ['purchase', 'login']),
 ('192.168.1.3', ['login'])]

In [41]:
category_log = log_rdd.groupBy(lambda x: x[0]) \
    .mapValues(lambda values: len([item for item in values if item[1] == "login"])) \
    .collect()
category_log

[('192.168.1.1', 1), ('192.168.1.2', 1), ('192.168.1.3', 1)]

In [20]:
#학생 성적 데이터 분석 (과목별 평균 성적 계산)
# SparkContext 생성 가정 (sc)
student_grades = [
    ("Math", 85),
    ("English", 90),
    ("Math", 92),
    ("Science", 88),
    ("English", 95),
    ("Science", 82)
]

# RDD 생성
grades_rdd = sc.parallelize(student_grades)


In [25]:
sub_grade = grades_rdd.groupBy(lambda x: x[0])\
            .mapValues(lambda val: sum(i[1] for i in val))\
            .collect()

sub_grade

[('Science', 170), ('Math', 177), ('English', 185)]

# 5. aggregate 함수 사용 예제

<details>
<summary>필요성</summary>
✅ 먼저, 왜 aggregate()가 필요할까?   
보통 RDD에 reduce()를 쓰면 하나의 값으로 줄이는데, 두 가지 이상의 결과(예: 합계 + 개수)를 동시에 계산하고 싶을 때는 reduce()로는 부족해.   
그래서 튜플이나 구조화된 형태로 집계할 수 있는 aggregate()를 사용하는 거야.
</details>

<details>
<summary>개념설명</summary>
🧠 핵심 부분: 평균 계산   
python
```
sum_count = numbers.aggregate(
    (0, 0),  # 초기값 (합계, 개수)

    lambda acc, value: (acc[0] + value, acc[1] + 1),  # 파티션 내부 연산

    lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])  # 파티션 간 합산
)
```

🧩 쉽게 말해 이런 식으로 동작함:   
각 파티션에서:   
값을 하나씩 받아서 (합계, 개수)를 누적한다.

모든 파티션이 끝나면:   
각 파티션의 (합계, 개수) 결과를 전부 더해서 최종 합계와 개수를 만든다.

🔢 예시로 계산해보자
```
RDD = [1, 5, 3, 9, 2, 8, 4, 7, 6]
3개의 파티션 예시 (내부 분배는 Spark가 결정):

P0: [1, 5, 3] → 합: 9, 개수: 3

P1: [9, 2, 8] → 합: 19, 개수: 3

P2: [4, 7, 6] → 합: 17, 개수: 3

최종 합산:
```
```
python
(9 + 19 + 17, 3 + 3 + 3) = (45, 9)
✅ 최종 결과
sum_count = (45, 9)
→ 평균은 직접 계산하면 돼:
```
```
python
avg = sum_count[0] / sum_count[1]  # 45 / 9 = 5.0
```
📌 핵심 요약
| 용도 | 동시에 여러 개의 결과값을 누적할 때 |
| 함수 구조 | aggregate(초기값, 파티션 내 연산, 파티션 간 합산) |
| 활용 예시 | 평균, 표준편차, 누적 count 등 |
| 이점 | reduce()보다 유연하며 구조화된 데이터 처리에 적합 |

🎯 비유로 설명하자면…
회사의 3개 부서에서 월급 총합과 인원수를 각각 계산하고

나중에 부서별 결과를 합산해서 전체 평균 월급을 구하는 방식과 같아


</details>

In [42]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
numbers = sc.parallelize([1, 5, 3, 9, 2, 8, 4, 7, 6], numSlices=3)
numbers.collect()

[1, 5, 3, 9, 2, 8, 4, 7, 6]

In [ ]:
# glom()은 각 파티션의 내용을 리스트로 감싸서 반환
# 즉, 파티션 간의 이동이 없고, 내부 자료구조만 바꾸는 연산
# glom()은 RDD의 각 파티션을 배열 형태로 변환하는 narrow transformation

In [43]:

numbers.glom().collect()

[[1, 5, 3], [9, 2, 8], [4, 7, 6]]

In [ ]:

numbers = sc.parallelize([1, 5, 3, 9, 2, 8, 4, 7, 6])

In [44]:
# 평균 계산하기
sum_count = numbers.aggregate(
    (0, 0),  # 초기값 (합계, 개수)
    lambda acc, value: (acc[0] + value, acc[1] + 1),  # 각 파티션 내 연산
    lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])  # 파티션 간 연산
)
sum_count

(45, 9)

🧠 전체 흐름: 3단계
🔹 1단계. RDD를 파티션으로 나눈다
RDD 데이터:

python
```
[1, 5, 3, 9, 2, 8, 4, 7, 6]
3개의 파티션으로 나뉘었을 때 (Spark가 자동으로 나누지만 예시):

P0: [1, 5, 3]

P1: [9, 2, 8]

P2: [4, 7, 6]
```
🔹 2단계. 각 파티션 내부에서 seqOp 실행
```
lambda acc, value: (acc[0] + value, acc[1] + 1)
이 함수는 각 파티션 안에서 값을 누적하는 역할을 한다.
초기값은 (0, 0)으로, 첫 번째는 합계, 두 번째는 개수를 의미한다.

▶ P0 파티션 [1, 5, 3]
시작: (0, 0)

(0 + 1, 0 + 1) → (1, 1)

(1 + 5, 1 + 1) → (6, 2)

(6 + 3, 2 + 1) → (9, 3)
```   
✅ 결과: (9, 3)

--
```
▶ P1 파티션 [9, 2, 8]
시작: (0, 0)

(0 + 9, 0 + 1) → (9, 1)

(9 + 2, 1 + 1) → (11, 2)

(11 + 8, 2 + 1) → (19, 3)
```   
✅ 결과: (19, 3)

--
```
▶ P2 파티션 [4, 7, 6]
시작: (0, 0)

(0 + 4, 0 + 1) → (4, 1)

(4 + 7, 1 + 1) → (11, 2)

(11 + 6, 2 + 1) → (17, 3)
```   
✅ 결과: (17, 3)

--
🔹 3단계. 파티션 간 결과 병합 (combOp 실행)
```
lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])
이 함수는 파티션들의 중간 결과를 최종적으로 합친다.

P0 + P1 → (9 + 19, 3 + 3) → (28, 6)

(28, 6) + P2 → (28 + 17, 6 + 3) → (45, 9)
```   
✅ 최종 결과
```
sum_count = (45, 9)
```   
즉,   

전체 합계: 45   
전체 개수: 9   
평균: 45 / 9 = 5.0




In [45]:
average = sum_count[0] / sum_count[1]
print("Average:", average)  # 5.0

Average: 5.0


In [46]:

# agg 연습
data = ["hello", "world", "spark"]
rdd = sc.parallelize(data)

def seq_op(acc, value):
    text, idx = acc
    return (f"{text} {idx}:{value}".strip(), idx + 1)

def comb_op(acc1, acc2):
    # 줄 번호 이어붙이기: 앞쪽 text 유지 + 뒤쪽 text 뒤에 붙이기
    text1, idx1 = acc1
    text2, idx2 = acc2
    # 줄 번호 충돌 피하려면 idx 조정 필요 (복잡하므로 단순히 붙임)
    return (f"{text1} {text2}".strip(), idx1 + idx2)

In [ ]:

zero = ("", 1)

result = rdd.aggregate(zero, seq_op, comb_op)[0]
print(result)

In [ ]:

rdd = sc.parallelize(["spark", "hadoop", "ai", "python", "sql"])
result = rdd.aggregate(zero, seq_op, comb_op)[0]
print(result)

In [ ]:
# 길이가 5 이상인 문자열 개수 세기
rdd = sc.parallelize(["spark", "hadoop", "ai", "python", "sql"])

result = rdd.aggregate(
    0,
    lambda acc, value: acc + (1 if len(value) >= 5 else 0),
    lambda acc1, acc2: acc1 + acc2
)

print("길이 5 이상인 문자열 개수:", result)

In [48]:
spark.stop() #Sparksession
sc.stop() #SparkContext